<a href="https://colab.research.google.com/github/saharmor/dalle-playground/blob/main/DALL_E_Playground_Backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><img src="https://em-content.zobj.net/thumbs/240/apple/325/artist_1f9d1-200d-1f3a8.png" width="120">
</center>

### <center>Use this notebook to run your text-to-image server</center>
### <center> [DALL-E Playground Repository](https://github.com/saharmor/dalle-playground) </center>

## Install dependencies

In [ ]:
!git clone https://github.com/saharmor/dalle-playground.git
!pip install -r dalle-playground/backend/requirements.txt
!pip install diffusers["torch"]

# Run the backend web server

In [ ]:
from threading import Thread, Event

app_port = 8000
announce_url = None
cloudflared_startup = Event()

def update_announce_url(url):
    global announce_url
    announce_url = url

def start_cloudflared(port):
  from flask_cloudflared import _run_cloudflared
  try:
    announce_url = _run_cloudflared(port, 8888)
  except:
    raise
  finally:
    update_announce_url(announce_url)
    cloudflared_startup.set()

def run_with_cloudflared(thread):
    old_run = thread.run

    def new_run(*args, **kwargs):
        new_thread = Thread(target = start_cloudflared, args=(app_port,))
        new_thread.setDaemon(True)
        new_thread.start()
        old_run(*args, **kwargs)

    thread.run = new_run

def app():
  !python dalle-playground/backend/app.py --port {app_port} --save_to_disk true --img_format jpeg --output_dir generations

if __name__ == '__main__':
    t1 = Thread(target = app)
    run_with_cloudflared(t1)
    t1.start()
    cloudflared_startup.wait()
    print(f"Your url is: {announce_url}")
    t1.join()

Your url is: https://vermont-threatened-guilty-customise.trycloudflare.com
--> Starting DALL-E Server. This might take up to two minutes.
Downloading: 100% 475/475 [00:00<00:00, 656kB/s]
Fetching 12 files:   0% 0/12 [00:00<?, ?it/s]
Downloading: 100% 340/340 [00:00<00:00, 411kB/s]
Fetching 12 files:  17% 2/12 [00:00<00:00, 10.51it/s]
Downloading: 100% 629/629 [00:00<00:00, 904kB/s]

Downloading:   0% 0.00/681M [00:00<?, ?B/s]
Downloading:   0% 513k/681M [00:00<02:13, 5.11MB/s]
Downloading:   0% 1.33M/681M [00:00<01:39, 6.86MB/s]
Downloading:   0% 2.40M/681M [00:00<01:20, 8.46MB/s]
Downloading:   1% 3.74M/681M [00:00<01:05, 10.4MB/s]
Downloading:   1% 5.48M/681M [00:00<00:52, 12.9MB/s]
Downloading:   1% 7.68M/681M [00:00<00:42, 16.0MB/s]
Downloading:   2% 10.5M/681M [00:00<00:33, 20.0MB/s]
Downloading:   2% 14.1M/681M [00:00<00:26, 25.0MB/s]
Downloading:   3% 18.7M/681M [00:00<00:21, 31.4MB/s]
Downloading:   4% 24.5M/681M [00:01<00:16, 39.9MB/s]
Downloading:   5% 31.2M/681M [00:01<00:13

### Now, take the url you got from cloudflare (should look like `your url is: https://xxxxxx.trycloudflare.com`), replace it within the url here https://saharmor.github.io/dalle-playground/?backendUrl=https://xxxxxx.trycloudflare.com and run it in the browser.

### Let the fun begin ✨